Todo: 

    - sacar las bool de las numéricas,
    - ver que le esta haciendo a las ord
    - DONE ver si tengo que usar el label encoder
    - ver bien los transformers (aplicar diferentes a var numericas?)
    - reentrenar

In [1]:
import os
os.chdir("..") 

In [2]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, make_union
from utils.Transformers2 import FillNaDict, DFFeatureUnion, ColumnSelector, DFStandardScaler, DummyTransformer, DFKBinsOrdinalDiscretizer, DFOrdinalEncoder
import numpy as np
from utils.MinMaxScalerOutliers import MinMaxScalerQ
from sklearn.model_selection import GridSearchCV
import joblib
from tqdm import tqdm


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [4]:
seed = 1111

In [5]:
data_path = "data/"

In [6]:
X_train = pd.read_csv(data_path + 'train.csv', sep='\t', encoding='utf-8', )  
X_train.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,is_official_store,catalog_product_id,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,address.state_name,shipping.free_shipping,shipping.mode,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name
0,12.0,70.17,MLA896914637,MLA,Magnetron Para Microondas Modelo 2m219g 2m 219 G,1850.0,ARS,1,50,buy_it_now,gold_special,new,1,MLA455062,0,NaN,0,169949346,0,0,22721.0,906.0,historic,0.01,0.98,0.01,21815.0,platinum,0.0053,0.0011,60 days,3587.0,0.0029,Buenos Aires,0,me2,cross_docking,0,NaN,MLA5726,Electrodomésticos y Aires Ac.,NaN,Magnetrones
1,12.0,70.17,MLA689849716,MLA,Motor Y Pedal P/máquinas De Coser Familiar Sin...,4890.0,ARS,1,250,buy_it_now,gold_special,new,1,MLA116430,0,NaN,0,93137649,0,0,4037.0,175.0,historic,0.00,1.00,0.00,3862.0,platinum,0.0000,0.0221,60 days,758.0,0.0000,Buenos Aires,1,me2,fulfillment,0,NaN,MLA5726,Electrodomésticos y Aires Ac.,NaN,Otros Accesorios
2,NaN,NaN,MLA909338268,MLA,Honda Civic 1.8 Exs At 140cv - 127218 - C(p),1480000.0,ARS,1,0,classified,gold_premium,used,1,MLA1744,0,MLA8713970,0,439841655,1,0,150.0,72.0,historic,0.02,0.93,0.05,78.0,NaN,0.0000,0.0000,365 days,78.0,0.0000,Capital Federal,0,not_specified,not_specified,0,NaN,MLA1743,"Autos, Motos y Otros",NaN,Autos y Camionetas
3,12.0,70.17,MLA750172551,MLA,Radio Yongnuo Rf-603 Ver Ii / Nikon O Canon / ...,6688.0,ARS,1,100,buy_it_now,gold_special,new,1,MLA411998,0,NaN,0,141498690,0,0,14212.0,786.0,historic,0.00,1.00,0.00,13426.0,platinum,0.0057,0.0102,60 days,2325.0,0.0000,Capital Federal,1,me2,fulfillment,0,NaN,MLA1039,Cámaras y Accesorios,NaN,Disparadores
4,NaN,NaN,MLA853999531,MLA,"1, 2 Y 3 Ambientes Con Cocheras Opcionales",110000.0,USD,1,0,classified,gold,new,0,MLA401806,0,NaN,0,102898425,0,1,0.0,0.0,historic,0.00,0.00,0.00,0.0,NaN,0.0000,0.0000,365 days,0.0,0.0000,Capital Federal,0,not_specified,not_specified,0,NaN,MLA1459,Inmuebles,NaN,Emprendimientos


In [7]:
X_train.shape

(23194, 43)

In [8]:
X_train.discount.describe()

count    2818.000000
mean        0.180847
std         0.119029
min         0.000000
25%         0.080000
50%         0.158469
75%         0.259146
max         0.720000
Name: discount, dtype: float64

In [58]:
X_train.available_quantity.value_counts()

1        13394
50        2604
500       2445
250       1515
100       1109
150        768
5000       698
200        424
50000      237
Name: available_quantity, dtype: int64

In [56]:
X_train.is_official_store.value_counts()

0    20961
1     2233
Name: is_official_store, dtype: int64

In [9]:
X_train.price.describe().round(2)

count       23009.00
mean       126819.38
std       1129333.64
min             1.00
25%           820.00
50%          1899.00
75%          5149.00
max      85518000.00
Name: price, dtype: float64

In [10]:
X_train['installments.rate'].describe().round(2)

count    21510.00
mean        63.82
std         20.14
min          0.00
25%         70.17
50%         70.17
75%         70.17
max         70.17
Name: installments.rate, dtype: float64

In [11]:
aux = X_train.copy()

## Target

In [12]:
X_train.sold_quantity.value_counts().sort_index()

0        2340
1         215
2         180
3         172
4         154
5        1768
25       1384
50       2108
100      1556
150      1326
200      1078
250      3493
500      6885
5000      528
50000       7
Name: sold_quantity, dtype: int64

<p style="color:red; font-weight: bold">Vamos a intentar probar unificar las categorias iniciales, considerando este tipo de decisiones, en caso de quen hagan que el modelo de mejor, deben consultarse siempre con el negocio </p>

In [13]:
X_train["sold_quantity"] = [x if x >= 5 else 1 for x in X_train["sold_quantity"]]

In [14]:
X_train.sold_quantity.value_counts().sort_index()

1        3061
5        1768
25       1384
50       2108
100      1556
150      1326
200      1078
250      3493
500      6885
5000      528
50000       7
Name: sold_quantity, dtype: int64

# y

In [15]:
y = X_train.sold_quantity

In [16]:
X_train = X_train.drop(['sold_quantity'], axis= 1)

## LabelEncoder

enc = LabelEncoder()
enc.fit(y)
y_transformed = enc.transform(y)

pd.Series(y_transformed).value_counts(normalize=True).sort_index().plot.barh()

## SMOTE strategy

In [17]:
dic = pd.Series(y).value_counts().sort_index().to_dict()
dic

{1: 3061,
 5: 1768,
 25: 1384,
 50: 2108,
 100: 1556,
 150: 1326,
 200: 1078,
 250: 3493,
 500: 6885,
 5000: 528,
 50000: 7}

In [18]:
threshold = 3000
prop = 1.5

In [19]:
smote_sampling_strategy = {key: int(value*prop) if value < threshold else value for key, value in dic.items()}
smote_sampling_strategy

{1: 3061,
 5: 2652,
 25: 2076,
 50: 3162,
 100: 2334,
 150: 1989,
 200: 1617,
 250: 3493,
 500: 6885,
 5000: 792,
 50000: 10}

## IDs  + columnas a descartar

In [20]:
id_cols = [
'id',
'site_id',
'title',
'category_id',
'seller.id',
'catalog_product_id',
'root_category_id',
'root_category_name',
'category_name',
]

In [21]:
columns_discard = [
    'tags',
    'seller.tags', 
    'shipping.tags',
    'categories_from_root', 
    'prices.prices',
    'seller.seller_reputation.metrics.delayed_handling_time.value',
    'seller.seller_reputation.metrics.delayed_handling_time.period',
    'seller.seller_reputation.metrics.cancellations.value',
    'seller.seller_reputation.metrics.cancellations.period',
    ]

In [22]:
ids_df = X_train[id_cols].copy()
X_train = X_train.drop(id_cols, axis= 1) # + columns_discard

In [23]:
X_train.dtypes

installments.quantity                                          float64
installments.rate                                              float64
price                                                          float64
currency_id                                                     object
available_quantity                                               int64
buying_mode                                                     object
listing_type_id                                                 object
condition                                                       object
accepts_mercadopago                                              int64
is_official_store                                                int64
catalog_listing                                                  int64
seller.car_dealer                                                int64
seller.real_estate_agency                                        int64
seller.seller_reputation.transactions.total                    float64
seller

## Columns

In [24]:
cols_preproc = X_train.columns
cols_preproc

Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_reputation.transactions.total',
       'seller.seller_reputation.transactions.canceled',
       'seller.seller_reputation.transactions.period',
       'seller.seller_reputation.transactions.ratings.negative',
       'seller.seller_reputation.transactions.ratings.positive',
       'seller.seller_reputation.transactions.ratings.neutral',
       'seller.seller_reputation.transactions.completed',
       'seller.seller_reputation.power_seller_status',
       'seller.seller_reputation.metrics.claims.rate',
       'seller.seller_reputation.metrics.delayed_handling_time.rate',
       'seller.seller_reputation.metrics.sales.period',
       'seller.seller_reputation.metrics.sales.completed',
       

In [25]:
categorical_ord_enc = [
    'available_quantity',
    'installments.quantity',
]

In [26]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).drop(categorical_ord_enc, axis = 1).columns.to_list()
numeric_features

['installments.rate',
 'price',
 'accepts_mercadopago',
 'is_official_store',
 'catalog_listing',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.sales.completed',
 'seller.seller_reputation.metrics.cancellations.rate',
 'shipping.free_shipping',
 'shipping.store_pick_up',
 'discount']

In [27]:
categorical_features = X_train.select_dtypes(include=['object']).columns.to_list() 
categorical_features

['currency_id',
 'buying_mode',
 'listing_type_id',
 'condition',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.sales.period',
 'address.state_name',
 'shipping.mode',
 'shipping.logistic_type',
 'seller.seller_reputation.real_level']

## NA

Nota: para las categóricas habría que consultar con expertos del dominio en cómo completar

In [28]:
fillNA = {
    'cat': {},
    'num': {},
    'ord': {}
}
for x in categorical_ord_enc:
    fillNA['ord'][x] = 0 

for x in numeric_features:
    fillNA['num'][x] = 0 
    
for x in categorical_features:
    fillNA['cat'][x] = '__NA__'

In [29]:
fillNA

{'cat': {'currency_id': '__NA__',
  'buying_mode': '__NA__',
  'listing_type_id': '__NA__',
  'condition': '__NA__',
  'seller.seller_reputation.transactions.period': '__NA__',
  'seller.seller_reputation.power_seller_status': '__NA__',
  'seller.seller_reputation.metrics.sales.period': '__NA__',
  'address.state_name': '__NA__',
  'shipping.mode': '__NA__',
  'shipping.logistic_type': '__NA__',
  'seller.seller_reputation.real_level': '__NA__'},
 'num': {'installments.rate': 0,
  'price': 0,
  'accepts_mercadopago': 0,
  'is_official_store': 0,
  'catalog_listing': 0,
  'seller.car_dealer': 0,
  'seller.real_estate_agency': 0,
  'seller.seller_reputation.transactions.total': 0,
  'seller.seller_reputation.transactions.canceled': 0,
  'seller.seller_reputation.transactions.ratings.negative': 0,
  'seller.seller_reputation.transactions.ratings.positive': 0,
  'seller.seller_reputation.transactions.ratings.neutral': 0,
  'seller.seller_reputation.transactions.completed': 0,
  'seller.sel

## PIPELINES

In [30]:
numeric_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict = fillNA['num'])),
    # ('binarizer',DFKBinsOrdinalDiscretizer(n_bins=5,cols=numeric_features))
    ])

categorical_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(categorical_features)),
    ('imputer', FillNaDict(col_fill_dict = fillNA['cat'])),
    ('dummies', DummyTransformer(categorical_features))
    ])


cat_label_encod = Pipeline(steps=[
    ('extractor',ColumnSelector(categorical_ord_enc)),
    ('imputer', FillNaDict(col_fill_dict = fillNA['ord'])),
    ('ordinalEncoder',DFOrdinalEncoder(categorical_ord_enc))
])

sel_ini_transformer = Pipeline(steps=[
    ('extractor',ColumnSelector(cols_preproc))
])

# DFFeatureUnion
feat_un = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('cat_lab_enc',cat_label_encod),
            ('numeric',numeric_transformer),
            ])

# pipeline_base
pipeline_base = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un)])
pipeline_base

Pipeline(steps=[('seleccionFeatsIni',
                 Pipeline(steps=[('extractor',
                                  ColumnSelector(columns=Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_...
       'seller.seller_reputation.metrics.sales.completed',
       'seller.seller_reputation.metrics.cancellations.rate',
       'address.state_name', 'shipping.free_shipping', 'shipping.mode',
       'shipping.logistic_type', 'shipping.store_pick_up',
       'seller.seller_reputation.real_level', 'discount'],
      dtype='object')))])),
                ('final_features',
                 <utils.Transformers2.DFFeatureUnion object at 0x0000017993D10B20>)])

In [31]:
## Pipeline con MinMaxScaler para vars numéricas

In [32]:
numeric_transformer_w_scaler = Pipeline(steps=[
    ('extractor',ColumnSelector(numeric_features)),
    ('imputer',FillNaDict(col_fill_dict= fillNA['num'])),
    ('binarizer',MinMaxScalerQ(X_train, columns=numeric_features, q_min = 0, q_max = 0.75))
    ])

# DFFeatureUnion
feat_un2 = DFFeatureUnion([
            ('categoricals',categorical_transformer),
            ('cat_lab_enc',cat_label_encod),
            ('numeric',numeric_transformer_w_scaler),
            ])

# pipeline_w_num_scaler
pipeline_w_num_scaler = Pipeline(steps=[('seleccionFeatsIni',sel_ini_transformer),
                                ('final_features',feat_un2)])

In [33]:
pipeline_w_num_scaler

Pipeline(steps=[('seleccionFeatsIni',
                 Pipeline(steps=[('extractor',
                                  ColumnSelector(columns=Index(['installments.quantity', 'installments.rate', 'price', 'currency_id',
       'available_quantity', 'buying_mode', 'listing_type_id', 'condition',
       'accepts_mercadopago', 'is_official_store', 'catalog_listing',
       'seller.car_dealer', 'seller.real_estate_agency',
       'seller.seller_...
       'seller.seller_reputation.metrics.sales.completed',
       'seller.seller_reputation.metrics.cancellations.rate',
       'address.state_name', 'shipping.free_shipping', 'shipping.mode',
       'shipping.logistic_type', 'shipping.store_pick_up',
       'seller.seller_reputation.real_level', 'discount'],
      dtype='object')))])),
                ('final_features',
                 <utils.Transformers2.DFFeatureUnion object at 0x0000017993D10EE0>)])

## MODELS

In [34]:
models = {         
          'lr': LogisticRegression(),
          'xgb':XGBClassifier(),
          'mlp':MLPClassifier(),
}

In [35]:
hparams = {'xgb':{'max_depth':[5,7,10], 'gamma': [1,5,10,20], 'n_jobs':[7], 'learning_rate': [0.05, 0.1], 'n_estimators':[100, 200, 500],
                 'colsample_bytree': [0.2, 0.5], 'seed': [seed]},
     
          'lr': {'multi_class': ['ovr'], 'class_weight':['balanced'], 'C': [0.01, 0.1, 1, 10,], 'solver': ['saga',], 'random_state':[seed]},
    
          
          'mlp':{'solver':["adam", 'sgd'], 'hidden_layer_sizes' : [(64, 64, 64), (64, 64, 64, 64), (64, 64, 64, 64, 64)], 'activation' : ['relu'], 'learning_rate_init' : [0.001, 0.01, 0.05], 'alpha' : [0.0001, 0.05, 0.001],
                 'max_iter':[1800], 'batch_size':[64, 256], 'learning_rate': ['constant','adaptive'], 'random_state':[seed]},         
     
}

In [36]:
diccionario_entrenamiento = {
                            'xgb': {},
                            'lr': {},
                            'mlp': {},
}
                                                         
for valor in diccionario_entrenamiento.keys():
    diccionario_entrenamiento[valor]['model'] = models[valor]
    diccionario_entrenamiento[valor]['params'] = hparams[valor]


## TRAINING

In [37]:
X_train = X_train.reset_index(drop=True)

In [38]:
cv_n = 3 # para aprovechar toda la data ... son muy pocos registros!

In [39]:
models_path = data_path + "/models/"


In [40]:
pipeline = pipeline_w_num_scaler # pipeline_base

In [41]:
Y_train_copy = y.copy()
X_train_copy = X_train.copy()

In [42]:
X_train_copy = pipeline.fit_transform(X_train_copy) 
X_train_copy.reset_index(drop=True, inplace= True)


## corrMatrix - post pipeline

selected_columns = [
    
]

corrMatrix = df[selected_columns].corr()
corrMatrix

sn.heatmap(corrMatrix, annot=True)
plt.show()

In [43]:
path_pipeline = data_path + 'pipeline.joblib'
joblib.dump(pipeline, path_pipeline)


['data/pipeline.joblib']

In [44]:
X_train_copy.head()

,shipping.logistic_type___NA__,shipping.logistic_type_cross_docking,shipping.logistic_type_custom,shipping.logistic_type_default,shipping.logistic_type_drop_off,shipping.logistic_type_fulfillment,shipping.logistic_type_not_specified,shipping.logistic_type_xd_drop_off,listing_type_id_bronze,listing_type_id_gold,listing_type_id_gold_premium,listing_type_id_gold_pro,listing_type_id_gold_special,listing_type_id_silver,seller.seller_reputation.metrics.sales.period_365 days,seller.seller_reputation.metrics.sales.period_60 days,seller.seller_reputation.metrics.sales.period_60 months,seller.seller_reputation.metrics.sales.period___NA__,seller.seller_reputation.transactions.period___NA__,seller.seller_reputation.transactions.period_historic,condition___NA__,condition_new,condition_not_specified,condition_used,currency_id_ARS,currency_id_USD,currency_id___NA__,buying_mode_buy_it_now,buying_mode_classified,address.state_name_Bs.As. Costa Atlántica,address.state_name_Bs.As. G.B.A. Norte,address.state_name_Bs.As. G.B.A. Oeste,address.state_name_Bs.As. G.B.A. Sur,address.state_name_Buenos Aires,address.state_name_Buenos Aires Interior,address.state_name_Capital Federal,address.state_name_Catamarca,address.state_name_Chaco,address.state_name_Chubut,address.state_name_Corrientes,address.state_name_Córdoba,address.state_name_Entre Ríos,address.state_name_Formosa,address.state_name_Jujuy,address.state_name_La Pampa,address.state_name_La Rioja,address.state_name_Mendoza,address.state_name_Misiones,address.state_name_Neuquén,address.state_name_Río Negro,address.state_name_Salta,address.state_name_San Juan,address.state_name_San Luis,address.state_name_Santa Fe,address.state_name_Santiago del Estero,address.state_name_Tierra del Fuego,address.state_name_Tucumán,address.state_name_Uruguay,seller.seller_reputation.power_seller_status___NA__,seller.seller_reputation.power_seller_status_gold,seller.seller_reputation.power_seller_status_platinum,seller.seller_reputation.power_seller_status_silver,seller.seller_reputation.real_level___NA__,seller.seller_reputation.real_level_green,seller.seller_reputation.real_level_green_gold,seller.seller_reputation.real_level_green_platinum,seller.seller_reputation.real_level_green_silver,seller.seller_reputation.real_level_orange,seller.seller_reputation.real_level_red,seller.seller_reputation.real_level_yellow,shipping.mode_custom,shipping.mode_me1,shipping.mode_me2,shipping.mode_not_specified,available_quantity,installments.quantity,installments.rate,price,accepts_mercadopago,is_official_store,catalog_listing,seller.car_dealer,seller.real_estate_agency,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,shipping.free_shipping,shipping.store_pick_up,discount
0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,0.366943,1.0,0.0,0.0,0.0,0.0,0.576251,0.505863,0.5,0.989899,0.5,0.581609,0.53,0.057292,0.518877,0.644444,0.0,0.0,0.0
1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,0.969919,1.0,0.0,0.0,0.0,0.0,0.102387,0.097711,0.0,1.000000,0.0,0.102965,0.00,1.000000,0.109648,0.000000,1.0,0.0,0.0
2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0.0,1.000000,1.0,0.0,0.0,1.0,0.0,0.003804,0.040201,1.0,0.939394,1.0,0.002080,0.00,0.000000,0.011283,0.000000,0.0,0.0,0.0

In [45]:
X_train_copy.discount.value_counts()

0.000000    20377
0.050000      253
0.100000      122
0.050000       87
0.060000       56
            ...  
0.152371        1
0.300060        1
0.200046        1
0.100022        1
0.102102        1
Name: discount, Length: 1466, dtype: int64

In [59]:
X_train_copy['installments.quantity'].value_counts()

0    24048
2     1753
1     1749
3      471
4       50
Name: installments.quantity, dtype: int64

In [47]:
X_train_copy['price'].value_counts()

1.000000    5800
0.000000     185
0.198149     153
0.694216     135
0.396496     131
            ... 
0.049240       1
0.085279       1
0.043240       1
0.069412       1
0.628255       1
Name: price, Length: 4393, dtype: int64

In [57]:
X_train_copy.available_quantity.value_counts()

0    16874
4     3049
3     2838
2     1723
1     1203
7      875
5      753
6      486
8      270
Name: available_quantity, dtype: int64

In [55]:
X_train_copy.is_official_store.value_counts()

0.000000    25393
1.000000     2455
0.936369        1
0.658240        1
0.077141        1
            ...  
0.509499        1
0.079113        1
0.204081        1
0.583497        1
0.460691        1
Name: is_official_store, Length: 225, dtype: int64

## smote

In [48]:
oversample = SMOTE(n_jobs=7,
                  sampling_strategy= smote_sampling_strategy)
X_train_copy, Y_train_copy = oversample.fit_resample(X_train_copy, Y_train_copy)


Instancias por clase post resampling:

In [49]:
Y_train_copy.value_counts().sort_index()

1        3061
5        2652
25       2076
50       3162
100      2334
150      1989
200      1617
250      3493
500      6885
5000      792
50000      10
Name: sold_quantity, dtype: int64

In [50]:
columns_names = X_train_copy.columns
columns_names.tolist()

['shipping.logistic_type___NA__',
 'shipping.logistic_type_cross_docking',
 'shipping.logistic_type_custom',
 'shipping.logistic_type_default',
 'shipping.logistic_type_drop_off',
 'shipping.logistic_type_fulfillment',
 'shipping.logistic_type_not_specified',
 'shipping.logistic_type_xd_drop_off',
 'listing_type_id_bronze',
 'listing_type_id_gold',
 'listing_type_id_gold_premium',
 'listing_type_id_gold_pro',
 'listing_type_id_gold_special',
 'listing_type_id_silver',
 'seller.seller_reputation.metrics.sales.period_365 days',
 'seller.seller_reputation.metrics.sales.period_60 days',
 'seller.seller_reputation.metrics.sales.period_60 months',
 'seller.seller_reputation.metrics.sales.period___NA__',
 'seller.seller_reputation.transactions.period___NA__',
 'seller.seller_reputation.transactions.period_historic',
 'condition___NA__',
 'condition_new',
 'condition_not_specified',
 'condition_used',
 'currency_id_ARS',
 'currency_id_USD',
 'currency_id___NA__',
 'buying_mode_buy_it_now',
 'b

In [51]:
path_columns = data_path + 'cols.joblib'
joblib.dump(columns_names, path_columns)

['data/cols.joblib']

In [52]:
models_path_template = models_path + '{}.joblib'

In [53]:
basic_opt_params = {'scoring': 'f1_macro', 'cv': cv_n, 'n_jobs': 7}

In [54]:
for valor in tqdm(diccionario_entrenamiento.keys()):
    model = diccionario_entrenamiento[valor]['model']
    params = diccionario_entrenamiento[valor]['params']
    
    opt = GridSearchCV(estimator=model, param_grid=params, )
    opt.set_params(**basic_opt_params)
    fit = opt.fit(X_train_copy[columns_names], Y_train_copy)
    
    # persisto el best estimator para cada uno
    model_name = models_path_template.format(valor)
    if valor == 'xgb':
        fit.best_estimator_.save_model(model_name)
    else:
        joblib.dump(fit.best_estimator_, model_name)

  0%|                                                                                                                                  | 0/3 [00:00<?, ?it/s]c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:43:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 33%|██████████████████████████████████████▎                                                                            | 1/3 [2:55:33<5:51:06, 10533.38s/it]c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\a309549\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag